In [1]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#load correlation matrix
import pickle
import numpy as np
all_corr = pickle.load(open('/content/drive/My Drive/correlation_matrix/correlations_filecc400.pkl', 'rb'))
print(np.shape(all_corr['Caltech_0051456'][0]))
print(np.shape(all_corr['Caltech_0051456'][1]))

(76636,)
()


In [3]:
#prepair dataset as input for NN model
import pandas as pd
from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

X = []
for ff in all_corr:
  X.append(list(all_corr[ff][0]))
X = pd.DataFrame(X)
sc = StandardScaler()
X = sc.fit_transform(X)
Y = []
for jj in all_corr:
  Y.append(all_corr[jj][1])
Y = np.asarray(Y)

print(np.shape(X))
print(np.shape(Y))

(884, 76636)
(884,)


In [7]:
#make a NN model
import keras
classifier = Sequential()
classifier.add(Dense(1550, activation='relu', kernel_initializer='random_normal', input_dim=76636))
# classifier.add(Dense(13, activation='relu', kernel_initializer='random_normal'))
# classifier.add(Dense(2, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
loss_fn = tf.keras.losses.MeanSquaredError(reduction='sum_over_batch_size')

classifier.compile(optimizer ='adam',loss=loss_fn, metrics =['accuracy'])

accuracy = []
F1 = []
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.10)

In [8]:
#fit model
classifier.fit(x_train,y_train, batch_size=40, epochs= 100)

Epoch 1/100
20/20 [==============================] - 1s 33ms/step - loss: 0.4654 - accuracy: 0.5321
Epoch 2/100
20/20 [==============================] - 1s 33ms/step - loss: 0.4398 - accuracy: 0.5597
Epoch 3/100
20/20 [==============================] - 1s 33ms/step - loss: 0.4298 - accuracy: 0.5698
Epoch 4/100
20/20 [==============================] - 1s 32ms/step - loss: 0.4321 - accuracy: 0.5673
Epoch 5/100
20/20 [==============================] - 1s 32ms/step - loss: 0.4364 - accuracy: 0.5635
Epoch 6/100
20/20 [==============================] - 1s 32ms/step - loss: 0.4268 - accuracy: 0.5723
Epoch 7/100
20/20 [==============================] - 1s 32ms/step - loss: 0.4274 - accuracy: 0.5698
Epoch 8/100
20/20 [==============================] - 1s 33ms/step - loss: 0.4105 - accuracy: 0.5887
Epoch 9/100
20/20 [==============================] - 1s 32ms/step - loss: 0.4061 - accuracy: 0.5937
Epoch 10/100
20/20 [==============================] - 1s 32ms/step - loss: 0.4010 - accuracy: 0.5975

In [9]:
#calculate accuracy & f1 in test dataset
y_pred=classifier.predict(x_test)

y_pred = np.where(y_pred > 0.5, 1, 0)

cm = confusion_matrix(y_test, y_pred)
accuracy.append((cm[0][0] + cm[1][1])/sum(sum(cm)))
precision = cm[0][0]/(cm[0][0] + cm[1][1])
recall =  cm[0][0]/(cm[0][0] + cm[1][0])
F1.append(2 * precision * recall / (precision + recall))

print('accuracy =', np.mean(accuracy))
print('F1 =', np.mean(F1))

accuracy = 0.6067415730337079
F1 = 0.47191011235955055


In [21]:
#get weights from model
for layer in classifier.layers:
    weights = layer.get_weights() # list of numpy arrays

In [ ]:
# np.shape(classifier.layers[0].get_weights()[1])